In [1]:
!nvidia-smi

Tue May 25 01:05:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prerequirement (install library and download data)

In [2]:
!pip install fairseq==v0.9.0
!pip install git+https://github.com/microsoft/fastseq.git

#!git clone https://github.com/NVIDIA/apex
#!sed -i 's/if (bare_metal_major != torch_binary_major) or (bare_metal_minor != torch_binary_minor):/if False:/' ./apex/setup.py
#!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

     |████████████████████████████████| 307kB 28.4MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp37-cp37m-linux_x86_64.whl size=2122224 sha256=0ddc8b57adfeca6b8f976be2c1524f3220a14485593cb4858b6491512086fe3f
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
  Cloning https://github.com/microsoft/fastseq.git to /tmp/pip-req-build-k960o0y_
  Running command git clone -q https://github.com/microsoft/fastseq.git /tmp/pip-req-build-k960o0y_
     |████████████████████████████████| 143kB 33.9MB/s 
     |████████████████████████████████| 1.2MB 42.1MB/s 
     |████████████████████████████████| 133kB 49.2MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 7.6MB 54.6MB/s 
  Created wheel for fastseq: filename=fastseq-0.0.4-cp37-cp37m-linux_x86_64.whl size=2045769 sha256=45345756fa195425399ef7befcabd

In [3]:
!mkdir -p data
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/for_bart/test.source -P data/
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/for_bart/test.target -P data/

!mkdir -p data/bin
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/test.source-target.source.bin -P data/bin/
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/test.source-target.source.idx -P data/bin/
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/test.source-target.target.bin -P data/bin/
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/test.source-target.target.idx -P data/bin/

!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/dict.source.txt -P data/bin/
!wget https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/len-1024.bin/dict.target.txt -P data/bin/


--2021-05-25 01:07:00--  https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/for_bart/test.source
Resolving fastseq.blob.core.windows.net (fastseq.blob.core.windows.net)... 52.239.237.36
Connecting to fastseq.blob.core.windows.net (fastseq.blob.core.windows.net)|52.239.237.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45641783 (44M) [text/plain]
Saving to: ‘data/test.source’

test.source         100%[===================>]  43.53M  19.8MB/s    in 2.2s    

2021-05-25 01:07:03 (19.8 MB/s) - ‘data/test.source’ saved [45641783/45641783]

--2021-05-25 01:07:03--  https://fastseq.blob.core.windows.net/data/tasks/cnn_dm/for_bart/test.target
Resolving fastseq.blob.core.windows.net (fastseq.blob.core.windows.net)... 52.239.237.36
Connecting to fastseq.blob.core.windows.net (fastseq.blob.core.windows.net)|52.239.237.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3559525 (3.4M) [text/plain]
Saving to: ‘data/test.target’

test.target

In [4]:
!wget 'https://dl.fbaipublicfiles.com/fairseq/models/bart.large.cnn.tar.gz'
!tar xvzf 'bart.large.cnn.tar.gz'

--2021-05-25 01:07:09--  https://dl.fbaipublicfiles.com/fairseq/models/bart.large.cnn.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3715581256 (3.5G) [application/x-tar]
Saving to: ‘bart.large.cnn.tar.gz’

bart.large.cnn.tar. 100%[===================>]   3.46G  21.4MB/s    in 2m 47s  

2021-05-25 01:09:56 (21.2 MB/s) - ‘bart.large.cnn.tar.gz’ saved [3715581256/3715581256]

bart.large.cnn/
bart.large.cnn/NOTE
bart.large.cnn/model.pt
bart.large.cnn/dict.source.txt
bart.large.cnn/dict.target.txt


# Using EL-Attention in Command Line Tool

Inference with EL-Attention

In [5]:
!time fastseq-generate-for-fairseq data/bin/ --path bart.large.cnn/model.pt  --use-el-attn --fp16 --task translation --batch-size 256 --gen-subset test --truncate-source --bpe gpt2 --beam 4 --min-len 55 --max-len-b 140 --no-repeat-ngram-size 3 --lenpen 2.0 --skip-invalid-size-inputs-valid-test > bart.el-attention_256.txt
# uncomment above line to run
# Expected time: (real	24m23.314s for inference on T5 in colab)

WARNING 2021-05-25 01:11:15,116 /usr/local/lib/python3.7/dist-packages/fastseq/models/__init__.py:17] transformers can not be imported.
INFO 2021-05-25 01:11:15,144 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/fairseq/beam_search_optimizer.py:28] Using Efficient-Lossless Attention optimization
/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:144: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
INFO 2021-05-25 01:11:15,332 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/fairseq/__init__.py:55] fairseq(v0.9.0) has been optimized by fastseq(v0.0.4).
WARNING 2021-05-25 01:11:15,333 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/transformers/__init__.py:57] transformers can not be imported. Please ignore this warning if you are not using transformers
tcmalloc: large alloc 1625169920 bytes == 0x5563b95cc000 @

Inference with MultiHead-Attention

In [6]:
# !time fairseq-generate data/bin/ --path bart.large.cnn/model.pt --fp16 --task translation --batch-size 32 --gen-subset test --truncate-source --bpe gpt2 --beam 4 --min-len 55 --max-len-b 140 --no-repeat-ngram-size 3 --lenpen 2.0 --skip-invalid-size-inputs-valid-test > bart.multihead-attention_32.txt
# uncomment above line to run
# fairseq 0.9.0 is compariable with Pytorch 1.6, but in colab its version is 1.8.1

In [7]:
!ls 

bart.el-attention_256.txt  bart.large.cnn.tar.gz  sample_data
bart.large.cnn		   data


In [8]:
!echo 'Inference speed when using el attention'
!tail -n 2 bart.el-attention_256.txt

!echo 'Inference speed when using multihead attention'
#!tail -n 2 bart.multihead-attention_32.txt

Inference speed when using el attention
| Translated 11490 sentences (944820 tokens) in 1472.9s (7.80 sentences/s, 641.48 tokens/s)
| Generate test with beam=4: BLEU4 = 17.18, 36.9/19.4/13.1/9.3 (BP=1.000, ratio=1.203, syslen=933330, reflen=776133)
Inference speed when using multihead attention


# Using EL-Attention in Python API (Include Caculating ROUGE)

## Inference

Removing '*--n 3200*' if want to inference on whole dataset. 

For best speed, using **fastseq-generate-for-fairseq ... --use-el-attn**

In [9]:
!wget https://raw.githubusercontent.com/microsoft/fastseq/EL-attention-doc/examples/EL-attention/summarize.py

--2021-05-25 01:36:33--  https://raw.githubusercontent.com/microsoft/fastseq/EL-attention-doc/examples/EL-attention/summarize.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3840 (3.8K) [text/plain]
Saving to: ‘summarize.py’

summarize.py        100%[===================>]   3.75K  --.-KB/s    in 0s      

2021-05-25 01:36:33 (63.8 MB/s) - ‘summarize.py’ saved [3840/3840]



In [10]:
!time python summarize.py \
    --model-dir bart.large.cnn/ \
    --model-file model.pt \
    --src data/test.source \
    --bsz 320 \
    --out 320_test.hypo \
    --use-el-attn \
    --n 3200

 # Expected time: (real	7m36.176s for inference 3200 samples on T5 in colab)

WARNING 2021-05-25 01:36:34,967 /usr/local/lib/python3.7/dist-packages/fastseq/models/__init__.py:17] transformers can not be imported.
INFO 2021-05-25 01:36:34,970 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/fairseq/beam_search_optimizer.py:28] Using Efficient-Lossless Attention optimization
/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:144: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
INFO 2021-05-25 01:36:35,150 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/fairseq/__init__.py:55] fairseq(v0.9.0) has been optimized by fastseq(v0.0.4).
WARNING 2021-05-25 01:36:35,151 /usr/local/lib/python3.7/dist-packages/fastseq/optimizer/transformers/__init__.py:57] transformers can not be imported. Please ignore this warning if you are not using transformers
loading archive file bart.large.cnn/
loading archive file 

In [11]:
!head 320_test.hypo

A French prosecutor says he is not aware of any video footage from on board the plane. German daily Bild and Paris Match claim to have found a cell phone video of the crash. A French Gendarmerie spokesman calls the reports "completely wrong" and "unwarranted" German airline Lufthansa says co-pilot Andreas Lubitz battled depression years before he took controls.
The Palestinian Authority becomes the 123rd member of the International Criminal Court. The formal accession was marked with a ceremony at The Hague, in the Netherlands. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
Amnesty International's annual report catalogs the use of state-sanctioned killing as a punitive measure. At least 607 people were executed around the world in 2014, compared to 778 in 2013. The U.S. remains one of the worst offenders for imposing capital punishment, with only Iran and Saudi Arabia

In [12]:
!cp 320_test.hypo test.hypo
!cp data/test.target test.target

## Evaluate

Follow https://github.com/pytorch/fairseq/tree/master/examples/bart to install *files2rouge*

In [13]:
!export CLASSPATH=/path/to/stanford-corenlp-full-2016-10-31/stanford-corenlp-3.7.0.jar

# Tokenize hypothesis and target files.
!cat test.hypo | java edu.stanford.nlp.process.PTBTokenizer -ioFileList -preserveLines > test.hypo.tokenized
!cat test.target | java edu.stanford.nlp.process.PTBTokenizer -ioFileList -preserveLines > test.hypo.target
!files2rouge test.hypo.tokenized test.hypo.target
# Expected output: (ROUGE-2 Average_F: 0.21227)

Error: Could not find or load main class edu.stanford.nlp.process.PTBTokenizer
Caused by: java.lang.ClassNotFoundException: edu.stanford.nlp.process.PTBTokenizer
Error: Could not find or load main class edu.stanford.nlp.process.PTBTokenizer
Caused by: java.lang.ClassNotFoundException: edu.stanford.nlp.process.PTBTokenizer
/bin/bash: files2rouge: command not found
